# Robyn Budget Allocator Demo

This notebook demonstrates how to use the Python implementation of Robyn's budget allocator.
It shows how to:
1. Load and prepare data
2. Configure the allocator
3. Run optimization scenarios
4. Analyze and visualize results

## Step 1: Load Exported R Data

In [ ]:
## Step 1: Setup and Import
import sys
import os
import pandas as pd
import numpy as np
from typing import Dict, Any, Union, List
import matplotlib.pyplot as plt

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

# Import necessary Robyn classes
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs
from robyn.data.entities.hyperparameters import Hyperparameters
from robyn.modeling.pareto.pareto_optimizer import ParetoResult
from utils.data_mapper import (
    load_data_from_json,
    import_input_collect,
    import_output_collect,
    import_output_models,
)

In [ ]:
# Load data from JSON exported from R
# raw_input_collect = load_data_from_json(
#     "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/dh_Pareto_InputCollect.json"
# )
# raw_output_collect = load_data_from_json(
#     "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/dh_Allocator_OutputCollect.json"
# )

raw_input_collect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/test_forAllocator_2000_iterations_5_trials_InputCollect.json"
)
raw_output_collect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/test_forAllocator_2000_iterations_5_trials_OutputCollect.json"
)

# raw_input_collect = load_data_from_json(
#     "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_InputCollect.json"
# )
# raw_output_collect = load_data_from_json(
#     "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputCollect.json"
# )
# raw_output_models = load_data_from_json(
#     "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputModels.json"
# )

# Convert R data to Python objects
r_input_collect = import_input_collect(raw_input_collect)
r_output_collect = import_output_collect(raw_output_collect)
# python_model_outputs = import_output_models(raw_output_models)

# Extract individual components
mmm_data = r_input_collect["mmm_data"]
featurized_mmm_data = r_input_collect["featurized_mmm_data"]
holidays_data = r_input_collect["holidays_data"]
hyperparameters = r_input_collect["hyperparameters"]
pareto_result = r_output_collect["pareto_result"]
# Print data summary
print(f"Data loaded successfully:")
print(
    f"- Data timeframe: {mmm_data.data[mmm_data.mmmdata_spec.date_var].min()} to {mmm_data.data[mmm_data.mmmdata_spec.date_var].max()}"
)
print(
    f"- Number of paid media channels: {len(mmm_data.mmmdata_spec.paid_media_spends)}"
)
print(f"- Channels: {mmm_data.mmmdata_spec.paid_media_spends}")

## Step 2: Set up Budget Allocator

Initialize the budget allocator with the selected model and data.

In [ ]:
# select_model = r_output_collect["pareto_result"].pareto_solutions[
#     0
# ]  # Taking first solution as example
# print(f"Selected model: {select_model}")

In [ ]:
for i in raw_output_collect["clusters"]["models"]:
    print(i["solID"])

In [ ]:
# Override
select_model = "3_216_3"

In [ ]:
r_pareto_result = r_output_collect["pareto_result"]

In [ ]:
import pandas as pd

# Set display options to show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# Assuming r_dt_hyppar is your DataFrame
r_dt_hyppar = r_pareto_result.result_hyp_param[
    r_pareto_result.result_hyp_param["solID"] == select_model
]
print(r_dt_hyppar)

## Step 3: Run Different Optimization Scenarios

### Scenario 1: Default Max Response

In [ ]:
print("Hyperparameters raw output:", hyperparameters)

In [ ]:
# Assuming `hyperparameters` is an instance of the `Hyperparameters` class

# Iterate over each channel and its hyperparameters
for channel, params in hyperparameters.hyperparameters.items():
    print(f"Channel: {channel}")
    print(f"  Thetas: {params.thetas}")
    print(f"  Shapes: {params.shapes}")
    print(f"  Scales: {params.scales}")
    print(f"  Alphas: {params.alphas}")
    print(f"  Gammas: {params.gammas}")
    print(f"  Penalty: {params.penalty}")
    print()

# Print other attributes of the Hyperparameters class
print(f"Adstock: {hyperparameters.adstock}")
print(f"Lambda: {hyperparameters.lambda_}")
print(f"Train Size: {hyperparameters.train_size}")

In [ ]:
print(hyperparameters.adstock.value)

In [ ]:
from robyn.allocator.entities.allocation_params import AllocatorParams
from robyn.allocator.entities.allocation_result import (
    AllocationResult,
    OptimOutData,
    MainPoints,
)
from robyn.allocator.entities.optimization_result import OptimizationResult
from robyn.allocator.entities.constraints import Constraints
from robyn.allocator.optimizer import BudgetAllocator
from robyn.allocator.constants import (
    SCENARIO_MAX_RESPONSE,
    ALGO_SLSQP_AUGLAG,
    CONSTRAINT_MODE_EQ,
    DEFAULT_CONSTRAINT_MULTIPLIER,
    DATE_RANGE_ALL,
)


# Create allocator parameters matching R Example 1
allocator_params = AllocatorParams(
    scenario=SCENARIO_MAX_RESPONSE,
    total_budget=None,  # When None, uses total spend in date_range
    target_value=None,
    date_range="all",
    channel_constr_low=[0.7],  # Single value for all channels
    channel_constr_up=[1.2, 1.5, 1.5, 1.5, 1.5],  # Different values per channel
    channel_constr_multiplier=3.0,
    optim_algo="SLSQP_AUGLAG",
    maxeval=100000,
    constr_mode=CONSTRAINT_MODE_EQ,
    plots=True,
)
# 0.7 is the lower bound
# 1.2 is the upper bound for the first channel, and so on
# $70 - $150
# let's say $100, budget allocator is about increasing decreasing media budget, advertiser wants to know how much to spend on each channel
# you cannot be infinitely increasing budget, so you have to set constraints
# 0.7 - 1.2 means you can increase by 20% or decrease by 30%
# Boundedx3 = 3x increase or decrease, so 90% lower bound, 150% upper bound.

print("\nInitial constraints:")
for channel, low, up in zip(
    mmm_data.mmmdata_spec.paid_media_spends,
    [0.7] * len(mmm_data.mmmdata_spec.paid_media_spends),  # Expand single value
    [1.2, 1.5, 1.5, 1.5, 1.5],  # Per channel values
):
    print(f"{channel}: {low:.1f}x - {up:.1f}x")

# Initialize budget allocator
max_response_allocator = BudgetAllocator(
    mmm_data=mmm_data,
    featurized_mmm_data=featurized_mmm_data,
    hyperparameters=hyperparameters,
    pareto_result=pareto_result,
    select_model=select_model,
    params=allocator_params,
)

## Step 3: Run Optimization
max_response_result = max_response_allocator.optimize()

In [ ]:
## Step 4: Analyze Results
print("\nOptimization Results Summary:")
print("-" * 50)
print(f"Model ID: {select_model}")
print(f"Scenario: {max_response_result.scenario}")
print(f"Use case: {max_response_result.usecase}")

results_df = pd.DataFrame(
    {
        "Channel": max_response_result.dt_optimOut.channels,
        "Initial Spend": max_response_result.dt_optimOut.init_spend_unit,
        "Optimized Spend": max_response_result.dt_optimOut.optm_spend_unit,
        "Spend Change %": (
            max_response_result.dt_optimOut.optm_spend_unit
            / max_response_result.dt_optimOut.init_spend_unit
            - 1
        )
        * 100,
        "Initial Response": max_response_result.dt_optimOut.init_response_unit,
        "Optimized Response": max_response_result.dt_optimOut.optm_response_unit,
        "Response Lift %": (
            max_response_result.dt_optimOut.optm_response_unit
            / max_response_result.dt_optimOut.init_response_unit
            - 1
        )
        * 100,
    }
)

print("\nDetailed Results:")
print(results_df.round(2))

# Print additional diagnostics
print("\nOptimization Parameters:")
print(f"Total budget: {max_response_allocator.constraints.budget_constraint:,.2f}")
print("Bound multiplier:", max_response_allocator.params.channel_constr_multiplier)
print("\nConstraint Violations:")
violations = np.sum(
    np.abs(
        max_response_result.dt_optimOut.optm_spend_unit
        - max_response_allocator.allocator_data_preparer.init_spend_unit
    )
)
print(f"Total allocation adjustment: {violations:,.2f}")

In [ ]:
print(max_response_result.dt_optimOut)
print(max_response_result.mainPoints)

In [ ]:
from robyn.visualization.allocator_visualizer import (
    AllocatorPlotter,
)
%load_ext autoreload
%autoreload 2
# Initialize plotter with just the essential data
plotter = AllocatorPlotter(
    allocation_result=max_response_result,
    budget_allocator=max_response_allocator
)

# Generate all plots
plots = plotter.plot_all(display_plots=False, export_location=None)

### Scenario 3: Target Efficiency
Optimize allocation based on target ROI/CPA.

In [ ]:
# from robyn.allocator.constants import (
#     SCENARIO_TARGET_EFFICIENCY,  # Import this instead of SCENARIO_MAX_RESPONSE
#     ALGO_SLSQP_AUGLAG,
#     CONSTRAINT_MODE_EQ,
#     DEFAULT_CONSTRAINT_MULTIPLIER,
#     DATE_RANGE_ALL,
# )

# # Create allocator parameters matching R Example 3 for target efficiency
# allocator_params = AllocatorParams(
#     scenario=SCENARIO_TARGET_EFFICIENCY,  # Change scenario
#     total_budget=None,  # When None, it will use all available dates
#     # target_value is optional - when None:
#     # - For revenue (ROAS): defaults to 0.8x of initial ROAS
#     # - For conversion (CPA): defaults to 1.2x of initial CPA
#     target_value=None,
#     date_range="all",
#     # Use default constraints
#     channel_constr_low=[
#         0.1
#     ],  # Lower constraint for target_efficiency typically starts at 0.1
#     channel_constr_up=[
#         10
#     ],  # Upper constraint for target_efficiency typically goes up to 10
#     channel_constr_multiplier=3.0,
#     optim_algo="SLSQP_AUGLAG",
#     maxeval=100000,
#     constr_mode=CONSTRAINT_MODE_EQ,
#     plots=True,
# )

# # Initialize and run allocator same as before
# target_efficiency_allocator = BudgetAllocator(
#     mmm_data=mmm_data,
#     featurized_mmm_data=featurized_mmm_data,
#     hyperparameters=hyperparameters,
#     pareto_result=r_output_collect["pareto_result"],
#     select_model=select_model,
#     params=allocator_params,  # or allocator_params_custom
# )

# # Run optimization
# target_efficiency_result = target_efficiency_allocator.optimize()

In [ ]:
# ## Step 4: Analyze Results
# print("\nOptimization Results Summary:")
# print("-" * 50)
# print(f"Model ID: {select_model}")
# print(f"Scenario: {target_efficiency_result.scenario}")
# print(f"Use case: {target_efficiency_result.usecase}")

# results_df = pd.DataFrame(
#     {
#         "Channel": target_efficiency_result.dt_optimOut.channels,
#         "Initial Spend": target_efficiency_result.dt_optimOut.init_spend_unit,
#         "Optimized Spend": target_efficiency_result.dt_optimOut.optm_spend_unit,
#         "Spend Change %": (
#             target_efficiency_result.dt_optimOut.optm_spend_unit
#             / target_efficiency_result.dt_optimOut.init_spend_unit
#             - 1
#         )
#         * 100,
#         "Initial Response": target_efficiency_result.dt_optimOut.init_response_unit,
#         "Optimized Response": target_efficiency_result.dt_optimOut.optm_response_unit,
#         "Response Lift %": (
#             target_efficiency_result.dt_optimOut.optm_response_unit
#             / target_efficiency_result.dt_optimOut.init_response_unit
#             - 1
#         )
#         * 100,
#     }
# )

# print("\nDetailed Results:")
# print(results_df.round(2))

# # Print additional diagnostics
# print("\nOptimization Parameters:")
# if target_efficiency_allocator.constraints.budget_constraint is not None:
#     print(
#         f"Total budget: {target_efficiency_allocator.constraints.budget_constraint:,.2f}"
#     )
# else:
#     print("Total budget: Unconstrained (Target Efficiency Mode)")
# print("Bound multiplier:", target_efficiency_allocator.params.channel_constr_multiplier)

# print("\nConstraint Violations:")
# violations = np.sum(
#     np.abs(
#         target_efficiency_result.dt_optimOut.optm_spend_unit
#         - target_efficiency_allocator.allocator_data_preparer.init_spend_unit
#     )
# )
# print(f"Total allocation adjustment: {violations:,.2f}")

In [ ]:
# print(target_efficiency_result.dt_optimOut)
# print(target_efficiency_result.mainPoints)

In [ ]:
# from robyn.visualization.allocator_visualizer import (
#     AllocatorPlotter,
# )
# %load_ext autoreload
# %autoreload 2
# # Initialize plotter with just the essential data
# plotter = AllocatorPlotter(
#     allocation_result=target_efficiency_result,
#     budget_allocator=target_efficiency_allocator
# )

# # Generate all plots
# plots = plotter.plot_all(display_plots=False, export_location=None)